
    ## 导学单元

    ### 课程目标
    - 理解Budyko约束如何嵌入到机器学习管线中以保持物理一致性。
    - 熟悉Notebook与脚本 `examples/example_budyko_ml_workflow.py` 的配合方式，便于在命令行与交互环境间切换。

    ### 前置知识
    - 基础 Python（NumPy/Pandas）与 Matplotlib/Seaborn 绘图经验。
    - 对能量-水量平衡、蒸散发与降水等基本水文气候概念的认知。

    ### 所需包和运行环境
    - Python 3.9+，建议在虚拟环境或 Conda 环境中运行 Jupyter Notebook。
    - 依赖包主要列于项目 `requirements.txt`，常用库包括 NumPy、Pandas、Matplotlib、Seaborn、Xarray、SciPy（按需）。
    - 如需加速或批量运行，可直接在命令行调用 `examples/example_budyko_ml_workflow.py` 以生成对应输出。

    ### 学习路径与章节骨架
    1. 背景/概念：理解Budyko约束如何嵌入到机器学习管线中以保持物理一致性。
    2. 数据加载与预览：理解数据来源、字段含义与基本统计特征。
    3. 核心方法步骤（配合 `examples/example_budyko_ml_workflow.py`）：结合脚本展示的数据分割、特征工程与约束损失设计，完成模型训练与评估。
    4. 结果可视化：通过学习曲线、误差对比和Budyko散点图检视模型是否遵守物理规律。
    5. 反思与扩展练习：尝试替换模型或损失函数，观察约束对泛化与物理一致性的影响。

    ### 你将学到什么
    - 在ML任务中实现Budyko约束或先验的常见方式。
- 如何平衡模型拟合精度与物理合理性。
- 利用Notebook记录实验超参数与评估指标，保证可重复性。

    ### 本节完成标志
    - [ ] 完成受Budyko约束的模型训练并生成评估与可视化输出。
- [ ] 能解释约束项对预测结果和物理一致性的影响。


# example_budyko_ml_workflow.py Notebook

Example workflow demonstrating the Budyko constrained ML integration.

In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from src.analysis.budyko_ml_workflow import run_budyko_ml_workflow
from src.visualization.budyko_plots import BudykoVisualizer


def _create_synthetic_catchment(catchment_id: str, seed: int) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    dates = pd.date_range("2000-01-01", "2011-12-31", freq="D")
    n = len(dates)

    precip = rng.gamma(shape=2.0, scale=2.0, size=n)
    pet = rng.gamma(shape=2.0, scale=1.6, size=n)
    discharge = np.clip(precip - pet * 0.5 + rng.normal(0, 0.5, size=n), 0, None)

    return pd.DataFrame(
        {
            "date": dates,
            "discharge_mm_day": discharge,
            "precip_mm_day": precip,
            "pet_mm_day": pet,
            "area_km2": np.full(n, 500.0),
        }
    )


def main() -> None:
    catchments = {
        "BASIN_A": _create_synthetic_catchment("BASIN_A", seed=1),
        "BASIN_B": _create_synthetic_catchment("BASIN_B", seed=2),
        "BASIN_C": _create_synthetic_catchment("BASIN_C", seed=3),
    }

    attributes = pd.DataFrame(
        {
            "catchment_id": list(catchments.keys()),
            "TC": [10.5, 12.1, 8.9],
            "SAI": [0.4, 0.6, 0.5],
            "NDVI": [0.45, 0.52, 0.49],
            "HFP": [15, 20, 11],
        }
    )

    workflow = run_budyko_ml_workflow(
        catchments,
        attributes,
        climate_columns=("P_mm_yr", "Ep_mm_yr"),
        perform_cross_validation=False,
    )

    print("Training table columns:", workflow.training_table.columns.tolist())
    print("\nModel training metrics:")
    for target, metrics in workflow.model.training_metrics_.items():
        print(f"  {target}: R²={metrics['r2']:.3f}, RMSE={metrics['rmse']:.3f}")

    print("\nPredicted parameters:")
    print(workflow.predictions[["alpha", "Qbp_mm_yr", "runoff_ratio", "baseflow_ratio"]])

    fig, ax = plt.subplots(figsize=(6, 5))
    plot_data = {
        "Mean": {
            "IA": workflow.predictions["Ep_mm_yr"] / workflow.predictions["P_mm_yr"],
            "IE": 1 - workflow.predictions["runoff_ratio"],
            "omega": workflow.predictions["alpha"].mean(),
            "ia_mean": (workflow.predictions["Ep_mm_yr"] / workflow.predictions["P_mm_yr"]).mean(),
            "ie_mean": (1 - workflow.predictions["runoff_ratio"]).mean(),
            "start_year": 2000,
            "end_year": 2011,
        }
    }
    BudykoVisualizer.plot_catchment_trajectory(ax, plot_data)
    plt.tight_layout()
    plt.show()


if __name__ == "__main__":
    main()